In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [5]:
import torch
from llm.transformer import TransformerLM
from llm.tokenization import Tokenizer
from llm.serialization import load_checkpoint
from llm.generation import generateLLM

In [51]:
tokenized_dataset_pkl = "/media/bryan/ssd01/expr/llm_from_scratch/tokenization/bpe_10k_tinystories.pkl"
# tokenized_dataset_pkl = "/media/bryan/ssd01/expr/llm_from_scratch/tokenization/bpe_32k_owt_train.pkl"
eos_token = "<|endoftext|>"
tokenizer = Tokenizer.from_pickle(tokenized_dataset_pkl, special_tokens=[eos_token])
vocab_size = len(tokenizer.vocab)

In [58]:
model_pth = "/media/bryan/ssd01/expr/llm_from_scratch/tune-batch-size/lr1e-3-bs256-iters50k/checkpoint_best.pt"
model = TransformerLM(
    vocab_size=vocab_size,
    context_length=256,
    num_layers=4,
    num_heads=16,
    d_model=512,
    d_ff=1344,
    rope_theta=10000,
)

# model_pth = "/media/bryan/ssd01/expr/llm_from_scratch/owl-model-size/medium-500k-lr5e-4/checkpoint_best.pt"
# model = TransformerLM(
#     vocab_size=vocab_size,
#     context_length=256,
#     num_layers=24,
#     num_heads=16,
#     d_model=1024,
#     d_ff=4096,
#     rope_theta=10000,
# )
checkpoint = torch.load(model_pth)
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [59]:
model.eval();
model.to("cuda");
model.to(torch.bfloat16)

TransformerLM(
  (token_embeddings): Embedding(vocab_size=10000, d=512)
  (RoPE): RotaryPositionalEmbedding(context_length=256, dim/2=16)
  (layers): ModuleList(
    (0-3): 4 x TransformerBlock(
      (attn): CausalMHSARoPE(
        (qkv_proj): Linear(d_out=1536, d_in=512)
        (output_proj): Linear(d_out=512, d_in=512)
        (RoPE): RotaryPositionalEmbedding(context_length=256, dim/2=16)
      )
      (ffn): SwiGLU(
        (w1): Linear(d_out=1344, d_in=512)
        (w2): Linear(d_out=512, d_in=1344)
        (w3): Linear(d_out=1344, d_in=512)
      )
      (ln1): RMSNorm(hidden_size=512, eps=1e-05)
      (ln2): RMSNorm(hidden_size=512, eps=1e-05)
    )
  )
  (ln_final): RMSNorm(hidden_size=512, eps=1e-05)
  (lm_head): Linear(d_out=10000, d_in=512)
)

In [70]:
PROMPTS = [
#     "Once upon a time there was a little boy named Ben. Ben loved to",
   "Once upon a time, there was a pretty girl named Lily. She loved to eat",
#     "Machine learning is a subset of artificial intelligence that focuses on building systems that learn from data. One of the key algorithms in supervised learning is"
]
for prompt in PROMPTS:
    generated_text = generateLLM(
        model,
        tokenizer,
        prompt,
        max_new_tokens=256,
        temperature=1,
        top_k=25,
        top_p=0,
        eos_token=eos_token,
        seed=11,
    )
    print_text = f"PROMPT:\n{prompt}\nGENERATED:\n{prompt}{generated_text}\n"
    print(print_text)

PROMPT:
Once upon a time, there was a pretty girl named Lily. She loved to eat
GENERATED:
Once upon a time, there was a pretty girl named Lily. She loved to eat pizza while she played games and shared her toys with her friends. One day, she decided to make a special menu for her friends.
Lily called her friends Tom and Lily and said, "Come see the menu! We have pizza!" Tom and Lily came and saw the menu. They all wanted to eat cheese for their lunch. Lily's mom gave them some pizza, and they all sat down to eat.
After they were done eating, Lily said, "I will record the yummy food on my plate. I am sure my friends will love it." She held her plate and started to record. They all smiled and said, "Thank you for the yummy food!" Lily was happy to share her yummy food with her friends.
<|endoftext|>

